In [7]:
# -*- encoding: utf8-*-#coding:utf-8
import requests
import time
import random
import json
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
#print(json.dumps(return_json, indent=2))

In [8]:
def get_top_songs():
    top_songs = []

    headers = {'Referer': 'https://kma.kkbox.com/charts/yearly/newrelease?lang=tc&terr=tw',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
                'Content-Type': 'application/json'}
    url = 'https://kma.kkbox.com/charts/api/v1/yearly?category=297&lang=tc&limit=100&terr=tw&type=newrelease&year=2021'
    
    return_data = requests.get(url, headers=headers)
    return_json = json.loads(return_data.text)

    for song in return_json['data']['charts']['newrelease']:
        top_songs.append(re.split(' -| \(', song['song_name'])[0])
    
    return top_songs

In [9]:
# 獲取樂譜頁面
def get_pu(song_name):

    url = 'https://www.91pu.com.tw/plus/search.php?keyword=' + song_name + '&type='
    print(url)
    return_html = requests.get(url)
    soup = BeautifulSoup(return_html.text, 'html.parser')
    pu_url = 'https://www.91pu.com.tw' + soup.select('tr > td > a')[0].get('href')

    return pu_url

In [25]:
def get_chords_and_lyrics(pu_url):
    chord, lyric = [], []
    options = Options()
    options.add_argument("--disable-notifications")
    options.headless = True
    driver = webdriver.Edge(options=options)
    driver.implicitly_wait(3)

    driver.get(pu_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    tone = soup.find(id='tone_z').find_all('p')
    # print(tone)
    clean = re.compile('<.*?>')
    nbsp = re.compile(u'\xa0+')
    for tags in tone:
        output = tags.text
        output = re.sub(clean, '', output)
        output = re.sub(nbsp, ' ', output)
        #print(output)
        
        classes = tags.get('class')
        if 'music' in classes:
            chord.append(output)
        elif 'lyric' in classes:
            lyric.append(output)
        else:
            print('Unknown class')
    driver.quit()
    return chord, lyric

In [26]:
# top_songs = get_top_songs()
chords = []
lyrics = []

for i in tqdm(range(10)):
    try:
        pu_url = get_pu(top_songs[i])
        #print(pu_url)
        chord, lyric = get_chords_and_lyrics(pu_url)
        #print(chord)
        #print(lyric)
        chords.append(chord)
        lyrics.append(lyric)
    except:
        chords.append(['no chords'])
        lyrics.append(['no lyrics'])


100%|██████████| 10/10 [04:56<00:00, 29.67s/it]


In [27]:

for i in tqdm(range(10)):
    print(i+1, '.', top_songs[i])
    print('------------------------------------')
    for j in range(len(chords[i])):
        print(chords[i][j])
        print(lyrics[i][j])
    print('------------------------------------')

100%|██████████| 10/10 [00:00<00:00, 455.44it/s]

1 . 愛情你比我想的閣較偉大
------------------------------------
[前奏] |Am |Am |G |G |F |F |C |C |
 
 |Dm |Dm |Am |Am |G |G |E |E |
 倒佇故鄉街市的中央 笑看世界咧轉踅
|Am |Am |G |G |
 你是我 烏暗的心唯一的光 親像是眼前的燈火
 |F |F |C |C |
 原諒我這款放蕩的人 原諒我做這款夭壽的夢
 |Dm |Dm |Am |Am |
 每一擺你的眼神咧振動 攏予我無法分西東 哦
 |G |G |E |E |
 我用我的人格來保證 我是永遠袂後悔 
|Am |Am |G |G | 
 你是我 目前為止的愛人內底 無疑是上水的彼個
 |F |F |C |C |
 我其實毋是毋願離開 攏怪你行入了我的人生
 |Dm |Dm |Am |Am |
 無心的愛情傷害已造成 你我著愛來負責任 哦
 |G |G |E |E |
 
 [間奏1] |Am |Am |G |G |F |F |C |C |
 
 |Dm |Dm |Am |Am |G |G |E |E |
★愛情你比我想的閣較偉大 你予我這世人最後的期待
 |Am |F |G |C |
 你是我的未來 希望你會了解
 |Bm7-5 |E |Am |Am |
 雖然江山易改 但本性難移 愛你愛甲白目眉
 |F |G |Em |Am |F |G |
 
[間奏2] |Am |Am |G |G |F |F |C |C |
(1)
 |Dm |Dm |Am |Am |G |G |E |E |
 
|Am |Am |G |G |F |F |C |C |
 倒佇故鄉街市的中央 笑看世界咧轉踅
 |Dm |Dm |Am |Am |G |G |E |E |
 我其實毋是毋願離開 攏怪你行入了我的人生 (回★)
 |F |G |Em |Am |F |G |
 
[尾奏] |Am |Am |G |G |F |F |C |C |
(2)
 |Dm |Dm |Am |Am |G |G |E |E |Am |Am |
 毋管你知毋知 予你全部的愛 愛你愛甲白目眉
------------------------------------
2 . 在這座城市遺失了你
-----------------------------

In [18]:
s = get_pu('阿拉斯加海灣')
print(s)
chord, lyric = get_chords_and_lyrics(driver, pu_url)
print(chord)
print(lyric)

https://www.91pu.com.tw/song/2021/0123/13381.html


MaxRetryError: HTTPConnectionPool(host='localhost', port=53960): Max retries exceeded with url: /session/b2e8554b882d9ffdc1c27006f097ccf6/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000266DF696920>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。'))